# Test logifoldv5


In [1]:
import tensorflow as tf
import keras
import sys, os
print(tf.__version__)
sys.path.append("/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/modules")
sys.path.append("/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/github/logifold")
import pickle
import pandas as pd
print('Modules are loaded. (local)')

dropbox_path = "/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/"
base_models_path = dropbox_path + 'Base_Models/'
adversarial_examples_path = dropbox_path + 'Adversarial_Examples/'
adversarial_models_path = dropbox_path + 'Adversarial_Models/'

test_logifold_path = dropbox_path + 'testfolder/logifold_test' + '/'

2.10.0
Modules are loaded. (local)


In [2]:
from keras.models import load_model

dropbox_path = "/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/"
base_models_path = dropbox_path + 'Base_Models/'
base_model = load_model(base_models_path+'CIFAR10models/ResNet/n_9_v1_cifar10.keras')

2025-04-25 01:27:12.744989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
(x, y), (x_test, y_test)= cifar10.load_data()
x_tr, x_v, y_tr, y_v = train_test_split(x, y, test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
x_train, x_val, y_train, y_val = x_tr,x_v,y_tr,y_v
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255

num_classes = 10

In [4]:
from keras.utils import to_categorical

In [5]:
# Shrink the size of data.
small_x_tr, small_y_tr = x_tr[:100] , y_tr[:100]
small_x_v, small_y_v = x_v[:100] , y_v[:100]
small_y_v = to_categorical(small_y_v,num_classes)

In [6]:
import collections
from typing import Any, Optional
import os 
import numpy as np
import pickle, importlib
import pandas as pd
import keras
import tensorflow as tf
import math


In [42]:
adv_x_tr = np.load(adversarial_examples_path + 'by_ResNet/' + "pgd_0.376_x_untarget.npy")
adv_x_val = np.load(adversarial_examples_path + 'by_ResNet/' + "pgd_0.376_x_val_untarget.npy")
adv_x_test = np.load(adversarial_examples_path + 'by_ResNet/'+"pgd_0.376_x_test_untarget.npy")


In [7]:


class TreeNode:
    def __init__(self, flat_target : set):
        '''
        In target tree, we can remove or add tree node in logifold.
        And each node, which is labeled by flattened target, there is corresponding targets and keys.
        
        For instance, model A has target [[2],[1,0]] with key (0,) and model B and C have target [[0],[1],[2]] with keys (1,) and (2,).
        Their targets are flattened to (0,1,2).
        They are stored as [(1,), (0,), (2,)] in self.keys.
        
        And self.target_to_keys = {
            ( (0,1), (2,) ) :  ( (0,), 0.8 ),
            ( (0,), (1,), (2,) ) : ( (1,), 0.9 ), ( (2,), 0.7 )
        }
        , self.targets = {
            ( (0,1), (2,) ), ( (0,), (1,), (2,) )
        }.
        '''
        self.flat_target : set = flat_target
        
        
        self.NextNodes : list[TreeNode] = []
        self.PrevNodes : list[TreeNode] = []
        
        # set of targets whose flattening is this flat target
        self.targets : set[tuple] = set() 
        # list of model keys corr. to this flat target. Union of target_to_keys. 
        self.keys : list[tuple] = [] 
        # dictionary { (thick) targets  -> model keys }
        self.target_to_keys : dict[tuple, list[tuple]] = collections.defaultdict(list) 
        
        
        self.refinement = Refinement()
        self.enumerated_refinement = tuple()
    def append_node(self, nextnode : "TreeNode"):
        self.NextNodes.append(nextnode)
        nextnode.PrevNodes.append(self)
        return
    def is_leaf(self):
        return len(self.NextNodes) == 0
    def is_root(self):
        return len(self.PrevNodes) == 0
    def __node__(self):
        return f'Node : {self.flat_target}'
    
    def _show_tree_from_this_node_(self, level=0):
        print("  " * level + repr(self.flat_target))
        for nextnode in self.NextNodes:
            nextnode._show_tree_from_this_node_(level + 1)
    
    def _update_refinement_(self):
        '''
        At each node, "targets" contains distict T_1, ... T_n sharing the same flattening, which is updated as new model added.
        However, we update the information of refinment only when it is needed. 
        Therefore the information 'targets' may differ from the up-to-date 'targets'.
        It could have removed one, or newly added one.
        '''
        
        # If up-to-date targets are merely bigger than before, it's easy to update its refinement.
        if self.refinement.targets.issubset(self.targets):
            difference = self.targets - self.refinement.targets
            self.refinement.update_refinement(difference)
        # Otherwise, we should check each refinement without the removed one.
        else:
            self.refinement = Refinement()
            self.refinement.update_refinement(self.targets)
                
            
             
class TargetTree:
    def __init__(self):
        self.nodes: list[TreeNode] = []
        self.flat_target_to_node : dict[tuple,TreeNode] = {}
        
    def add_node(self,
                 thick_targets : list,
                 flattened_target : tuple[tuple[int,],...],
                 model_key : tuple,
                 root : bool = False):
        '''
        thick_targets (list[list[int]]) : (Thick) Targets of model. It is supposed to be ordered.
        E.g., [[3],[2,1]] should be converted into [[1,2],[3]]. It'll be stored as ((1,2),(3,))
        flattened_target (tuple[tuple[int]]) : Each flattened target is of the form 1-length tuple, (t,).
        E.g., ( (1,), (2,), (3,) ). flatTuple(target) provides this format.
        model_key(tuple) : Model key.
        root (Optional[bool]) : False by default. True if you add a root node.
        '''
        
        
        
        # Assuming thick targets is sorted in a unique way, we need to conver it to hashable one.
        thick_targets_as_a_tuple = tuple(sorted(tuple(sorted(t)) for t in thick_targets))
        
        flattened_target_as_a_set = set(flattened_target)
        # if node exists in TargetTree
        if flattened_target in self.flat_target_to_node:
            node = self.flat_target_to_node[flattened_target]
            
            node.keys.append(model_key)
            
            node.targets.add(thick_targets_as_a_tuple)
            
            node.target_to_keys[thick_targets_as_a_tuple].append(model_key)
        # if node is new
        else:
            node = TreeNode(flattened_target_as_a_set)
            self.flat_target_to_node[flattened_target] = node
            node.keys.append(model_key)
            node.targets.add(thick_targets_as_a_tuple)
            node.target_to_keys[thick_targets_as_a_tuple].append(model_key)
            for other_node in self.nodes:
                if flattened_target_as_a_set.issubset(other_node.flat_target):
                    other_node.append_node(node)
                if other_node.flat_target.issubset(flattened_target_as_a_set):
                    node.append_node(other_node)
            if root:
                self.nodes.insert(0,node)
            else:
                self.nodes.append(node)
            
        
        
    def remove_node(self,thick_targets : list, flattened_target : tuple, model_key : tuple):
        '''
        thick_targets (list[list[int]]) : (Thick) Targets of model. It is supposed to be ordered.
        E.g., [[3],[2,1]] should be converted into [[1,2],[3]].
        flattened_target (tuple[tuple[int]]) : Each flattened target is of the form 1-length tuple, (t,).
        E.g., ( (1,), (2,), (3,) ). flatTuple(target) provides this format.
        model_key (tuple) : corresponding model key you'd like to remove from the tree.
        '''
        thick_targets_as_a_tuple = tuple(tuple(t) for t in thick_targets)
        print(f'Deleting {model_key} from the Target Tree...')
        if flattened_target in self.flat_target_to_node:
            node = self.flat_target_to_node[flattened_target]
            print(f'Corresponding {node.__node__()}', end = ' ')
            for key_and_accuracy in node.keys:
                if key_and_accuracy[0] == model_key:
                    node.keys.remove(key_and_accuracy)
                    node.target_to_keys[thick_targets_as_a_tuple].remove(key_and_accuracy)
                    print(f'{model_key} has been removed.')
                break
            if not node.target_to_keys[thick_targets_as_a_tuple]:
                node.targets.remove(thick_targets_as_a_tuple)
                print(f'{thick_targets} has been deleted. Because there is no other models corresponding to {thick_targets}')
            if not node.keys:
                self.flat_target_to_node.pop(flattened_target)
                prev_nodes = node.PrevNodes
                for prev_node in prev_nodes:
                    prev_node.NextNodes.remove(node)
                next_nodes = node.NextNodes
                for next_node in next_nodes:
                    next_node.PrevNodes.remove(node)
                print(f'{node.__node__()} is removed from tree because there is no corresponding model.')
        else:
            print(f"node corresponding to {flattened_target} doesn't exist in Target Tree.")
    def _print_tree_(self):
        if not self.nodes:
            print("No nodes")
        else:
            root = self.nodes[0]
            root._show_tree_from_this_node_()

class Refinement:
    """
    A class to maintain and update target refinements.
    """
    def __init__(self):
        # Enumerated targetes, [T_0, T_1, ...]
        self.enumerated_targets : list[tuple] = []
        
        # In order to check if it's up-to-date. It'll be compared with treenode.targetes
        self.targets : set[tuple] = set()
        
        self.combinations : list[tuple[int,...]] = []
        
        # mapping (valid) combinations to the corresponding refinement.
        self.valid_refinements : dict[tuple[int,...],tuple] = collections.defaultdict(tuple)
        # self.max_comb = max_combo
    
    def update_refinement(self,set_of_thick_targets : set[tuple]):
        """
    Update the refinement data given a set of newly added thick targets.
        """
        for thick_target in set_of_thick_targets:
            # thick_target is supposed to be new.
            self.enumerated_targets.append(thick_target)
            self.targets.add(thick_target)
            n = len(thick_target)
            
            if not self.combinations:
                new_combinations = [(i,) for i in range(n)]
                new_refinements = collections.defaultdict(tuple)
                for i in range(n):
                    new_refinements[(i,)] = (thick_target[i])
            else:
                new_combinations = []
                new_refinements = collections.defaultdict(tuple)
            while self.combinations:
                old_combination = self.combinations.pop()
                refinement = self.valid_refinements[old_combination]
                for i in range(n):
                    new_combinations.append(old_combination + (i,))
                    if refinement:
                        new_refinement = tuple(
                            sorted(tuple(
                            set.intersection(
                                set(refinement),set(thick_target[i])
                                )
                            ))
                            )
                        if new_refinement:
                            new_refinements[old_combination+(i,)] = new_refinement
            
            combo_count = len(new_combinations)
            # if combo_count > self.max_comb:
            #     raise RuntimeError(
            #         f"[Safety Break] Too many combinations ({combo_count}). "
            #         f"Max allowed is {self.max_comb}."
            #     )
            self.combinations = new_combinations
            self.valid_refinements = new_refinements
                
        
        

In [8]:
def flatTuple(target,ascending=True) -> tuple[tuple[int,...]]:
    
    out = ()
    for a in target:
        out+=tuple(sorted(a))
    if ascending:
        out = tuple(sorted(out))
    return out

In [137]:
flatTuple([[3,1,2],[5],[4]])

(1, 2, 3, 4, 5)

### Initialization

In [138]:
test = [0.1,0.3,0.2]

test.sort()
test

[0.1, 0.2, 0.3]

In [140]:
0 in [0.]

True

In [9]:
# Initialization
targetTree = TargetTree()
x = small_x_v
model_keys = []
epsilon = 1e-3
fuzdoms = collections.defaultdict(dict)
models = {}
raw_predictions : dict[tuple,np.ndarray]= collections.defaultdict(lambda : np.array([],dtype=float))
certain_parts : dict[tuple,dict[float,np.ndarray]]= collections.defaultdict(dict)
combined_answers : dict[TreeNode,dict[float,np.ndarray]] = collections.defaultdict(dict)
combined_answers_along_a_path : dict[tuple[TreeNode,...], dict[float,np.ndarray]] = {}

use_history = False
optimal_paths = []
optimal_thresholds = []
for node in targetTree.nodes:
    node._update_refinement_()
    

In [117]:
valid_paths : list[list[list[TreeNode]]]= []

In [10]:
thresholds = [0, 0.5,0.7310585786300049,0.8807970779778823,0.9525741268224334,0.9820137900379085,0.9933071490757153,0.9975273768433653,0.9990889488055994,0.9996646498695336,0.9998766054240137]


In [11]:
def inverse_sigmoid(x):
    x = round(math.log(x**(-1)-1),2)
    if x == -float('inf'):
        return x
    else:
        x = abs(x)
        return x
ths_for_fuzdom = {0 : -float('inf')}
for th in thresholds[1:]:
    ths_for_fuzdom[th] = inverse_sigmoid(th)
ths_for_fuzdom

{0: -inf,
 0.5: 0.0,
 0.7310585786300049: 1.0,
 0.8807970779778823: 2.0,
 0.9525741268224334: 3.0,
 0.9820137900379085: 4.0,
 0.9933071490757153: 5.0,
 0.9975273768433653: 6.0,
 0.9990889488055994: 7.0,
 0.9996646498695336: 8.0,
 0.9998766054240137: 9.0}

### import one model  : Base ResNet56v1

In [12]:
charts: dict[tuple, dict[str, Any]] = {}

In [13]:
base_model = load_model(base_models_path+'CIFAR10models/ResNet/n_9_v1_cifar10.keras')

### New Label For Target

In [14]:
def newLabelForTarget(x : np.ndarray , y: np.ndarray, target :list[list[int]],
    ):
    new_out = len(target)
    base = np.argmax(y, -1)
    out = np.full(base.size, -1, dtype=int)
    for new_lbl, t in enumerate(target):
        out[np.isin(base, t)] = new_lbl
    
    relabel = out
    kept_y = relabel != -1
    x, new_y = x[kept_y], relabel[kept_y]
    return x, new_y

In [143]:
test_x, test_y =  newLabelForTarget(small_x_tr, small_y_tr, target = [[0,1,2],[3,4],[5]])

In [148]:
test_y 

array([0, 2, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 2, 2, 1, 2, 1, 2, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 1, 1, 0, 0, 1, 2, 0, 2, 0, 1,
       2, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0])

In [15]:
charts[(0,)] = {
    'target': [[i] for i in range(10)],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet56v1, standard',
    'spec for loading model': None
}
models[(0,)] = base_model
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


### GetFuzDom

In [16]:
def getFuzDoms(key):
    model : keras.Model= models[key]
    fuzDom : dict = {}
    print("Computing fuzzy domains...")
    x = small_x_v
    y = small_y_v
    target = charts[key]['target']
    x_k, y_k = newLabelForTarget(x, y, target)
    predict = model.predict(x_k, batch_size=32, verbose=1)
    ans = np.argmax(predict, axis=-1)
    certainty = np.max(predict, axis=-1)
    threshold = -float('inf')
    mark = (ans == y_k)
    class_accs = []
    ind_cl = [ans == i for i in range(len(target))]
    y_cl = [y_k == i for i in range(len(target))]

    for i in range(len(target)):
        chosen_in_cl = ind_cl[i]
        if np.sum(chosen_in_cl) > 0:
            marked_class = ans[chosen_in_cl]==y_k[chosen_in_cl] # True positive
            class_accs.append(
                np.sum(marked_class)/len(marked_class)
                )
        else:
            class_accs.append(float('nan'))
    fuzDom[threshold] = (
        (np.sum(mark)/len(mark), 1.0),              # coverage is 100% for alpha = -inf
        np.array(class_accs),
        np.ones(len(target))               
    )
    threshold = 0.0
    total_len = len(y_k)
    minValNo = len(y) * 0.01
    while True:
        req_certainty = sigmoid(threshold)
        certain = (certainty > req_certainty)
        covered_len = np.sum(certain)
        if covered_len <= minValNo:
            break

        mark_certain = (ans[certain] == y_k[certain])
        overall_acc = np.mean(mark_certain)
        coverage_rate = covered_len / total_len

        class_accs = []
        class_coverages = []
        for i in range(len(target)):
            # coverage for each true class
            if np.sum(y_cl[i]) > 0:
                class_coverages.append(np.sum(certain & y_cl[i]) / np.sum(y_cl[i]))
            else:
                class_coverages.append(0.0)
            # accuracy for each predicted class
            chosen_in_cl = (certain & ind_cl[i])
            if np.sum(chosen_in_cl) > 0:
                mark_cl = ans[chosen_in_cl] == y_k[chosen_in_cl] # True Positive
                
                class_accs.append(np.mean(mark_cl))
            else:
                class_accs.append(float('nan'))

        fuzDom[threshold] = (
            (overall_acc, coverage_rate),
            np.array(class_accs),
            np.array(class_coverages)
        )
        threshold += 0.5
    charts[key]['fuzDom'] = fuzDom

In [17]:
getFuzDoms((0,))

Computing fuzzy domains...
4/4 [==============================] - 1s 66ms/step


In [18]:
target = [[i] for i in range(10)]
targetTree.add_node(target,
                    flatTuple(target),
                    (0,),
                    root=True
                    )

### Visualize Fuz dom

In [135]:
def vis_fuz_dom(k, accuracy = True, coverage = True):
    model_key = k
    data : dict = charts[model_key]['fuzDom']
    table_data = {}
    for k, v in data.items():
        if sigmoid(k) in thresholds:
            if accuracy and coverage:
                
                table_data[(sigmoid(k), 'acc')] = [np.round(v[0][0],3)]+[np.round(r,3) for r in v[1]]
                table_data[(sigmoid(k), 'cov')] = [np.round(v[0][1],3)]+[np.round(r,3) for r in v[2]]
            elif accuracy and not coverage:
                table_data[(sigmoid(k))] = [np.round(v[0][0],3)]+[np.round(r,3) for r in v[1]]
            elif not accuracy and coverage:
                table_data[(sigmoid(k))] = [np.round(v[0][1],3)]+[np.round(r,3) for r in v[2]]
    df = pd.DataFrame(table_data).T
    df.columns = ['all']+[f'{t}' for t in charts[model_key]['target']]
    return df

In [136]:
vis_fuz_dom((0,), accuracy=False, coverage=True)

,all,[0],[1],[2],[3],[4],[5],[6],[7],[8],[9]
0.000000,1.00,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
0.500000,0.97,1.000,1.000,1.000,0.714,1.000,1.000,1.000,1.000,1.000,0.875
0.731059,0.93,1.000,1.000,1.000,0.714,0.909,0.833,0.889,1.000,1.000,0.875
0.880797,0.89,1.000,1.000,1.000,0.714,0.818,0.667,0.778,1.000,1.000,0.875
0.952574,0.86,1.000,0.889,1.000,0.714,0.727,0.667,0.778,1.000,0.923,0.875
0.982014,0.82,0.933,0.889,1.000,0.714,0.727,0.667,0.667,0.778,0.923,0.875
0.993307,0.72,0.733,0.889,1.000,0.571,0.727,0.333,0.667,0.778,0.923,0.625
0.997527,0.66,0.667,0.889,0.714,0.429,0.636,0.333,0.667,0.667,0.923,0.625
0.999089,0.60,0.533,0.889,0.571,0.429,0.545,0.333,0.556,0.556,0.923,0.625
0.999665,0.54,0.467,0.667,0.571,0.429,0.364,0.250,0.556,0.556,0.923,0.625


### Train

In [26]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras import Model
from keras import layers
from keras.layers import (
    Dense)
# Concatenate, BatchNormalization, Dropout,
#     Conv2D, Flatten, LeakyReLU, Activation, add, AveragePooling2D, Input
# )


def train(
    self, key: tuple,
    *,
    x_tr=None, y_tr=None, x_v=None, y_v=None,
    new_key : tuple = (),
    active: bool = True,
    epochs: int = 50,
    lr: float = 1e-3,
    batch_size: int = 32,
    
    
    ini_th: float = 0.0,
    step: float = 0.5,
    min_val_percentage: float = 0.01,
    lr_schedule=None,
    save_each: bool = False,
    new_label: bool = True,
    monitor_coverage_level=None,
    verbose: int = 0,
    description: str | None = None,
    autosave: bool = True,
) -> str:
    """Fit key and return a short summary."""
    if new_key is not None:
        save_to_new = True
        
    model : Model = models[key]
    target = charts[key]["target"]
    filetype = charts[key]["filetype"]
    model_name = ''.join(f"_{i:03d}" for i in key)
    if len(target) < num_classes and new_label:
        x_tr, y_tr = newLabelForTarget(x_tr, y_tr, target)
        x_v, y_v   = newLabelForTarget(x_v, y_v, target)
      
    initial_description = str()
    
    if new_key is not None:
        charts[new_key]={}
        charts[new_key]['fuzDom'] = None
        charts[new_key]['target'] = charts[key]['target']
        charts[new_key]['active'] = active
        charts[new_key]['filetype'] = filetype
        
        #Since Keras-pretrained model saves like usual model, no spec is needed for loading the saved one.
        if charts[key]['spec for loading model'] is not None and charts[key]['spec for loading model']['name'] != 'Keras-pretrained':
            charts[new_key]['spec for loading model'] = charts[key]['spec for loading model']
        else:
            charts[new_key]['spec for loading model'] = None
        text = "Trained from Model %s.\n"%(model_name)
        if description is not None:
            text += description + "\n"
            initial_description += description + "\n"
        if charts[key]['description'] is not None:           
            text += "Description of Model %s: %s\n"%(model_name, 
                                                    charts[key]['description'])
            initial_description += "Description of Model %s: %s\n"%(model_name, 
                                                    charts[key]['description'])
        charts[new_key]['description'] = text
        
    else:
        new_key = key
    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["accuracy"]
    
    model.compile(optimizer=opt,
                loss="categorical_crossentropy",
                metrics=metrics)

    
    ckpt_path = ('.'+model_name+'.keras')

    # callbacks
    monitor = "val_accuracy" if monitor_coverage_level is None else "val_covermet"
    callbacks = [ModelCheckpoint(ckpt_path, monitor=monitor,
                                save_best_only=not save_each, verbose=verbose)]
    if lr_schedule:
        callbacks += [LearningRateScheduler(lr_schedule),
                    ReduceLROnPlateau(factor=np.sqrt(0.1), patience=5, min_lr=5e-7)]

    # fit 
    hist = model.fit(x_tr, y_tr, batch_size=batch_size,
                    validation_data=(x_v, y_v),
                    epochs=epochs, callbacks=callbacks, verbose=verbose)
    
    
    train_summary = str()
    getFuzDoms(iniTh=ini_th, step=step, 
                    minValPercentage=min_val_percentage, keys=[new_key], autosave=autosave)
    

    # ---------- summary ----------
    metric = "val_accuracy" if monitor_coverage_level is None else "val_covermet"
    best = np.max(hist.history[metric])
    first = hist.history[metric][0]
    return train_summary + f"best {metric} {best:.3f} (first {first:.3f})"


### Turn specialist

In [27]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 5, 10, 15, 18 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 15:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 5:
        lr *= 1e-1
#     print('Learning rate: ', lr)
    return lr


def turn_specialist(
        key: tuple,
        new_key: tuple,
        new_target: list,
        *,
        train: bool = True,
        x_v: np.ndarray | None = None,
        y_v: np.ndarray | None = None,
        x_tr: np.ndarray | None = None,
        y_tr: np.ndarray | None = None,
        epochs: int = 21,
        lr: float = 1e-3,
        batch_size: int = 128,
        lr_schedule : LearningRateScheduler | None =None,
        datagen=None,
        resample: bool = False,
        ini_th: float = 0.0,
        step: float = 0.5,
        min_val_percentage: float = 0.01,
        active: bool = True,
        save_each: bool = False,
        save_above_acc: float | None = None,
        save_bests: int | None = None,
        description: str = "",
        verbose: int = 0,
    ):
        
        
        chart = charts[key]
        model : Model = models[key]
        suffix = ''.join(f"_{i:03d}" for i in key)
        
        tag = f"model{suffix}"
        for w in model.weights:
            if tag not in w.name:
                w._handle_name = f"{w.name}_{tag}"
        for layer in model.layers:
            if tag not in layer.name:
                layer._name = f"{layer.name}_{tag}"

        # build specialist network
        base = Model(inputs = model.inputs, outputs = model.layers[-2].output, name=f"{new_key}_base")
        x    = keras.Input(shape=base.input_shape[1:], name=f"{new_key}_in")
        y    = Dense(len(new_target), name=f"{new_key}_dense")(base(x))
        z    = layers.Softmax(name=f"{new_key}_softmax")(y)
        specialist = Model(inputs = x, outputs = z)
        specialist.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        suffix = ''.join(f"_{i:03d}" for i in key)
        
        specialist.save(f"model{suffix}.keras")

        charts[new_key] = {
            "target": new_target,
            "active": active,
            "fuzDom": None,
            "filetype": chart["filetype"],
            "spec for loading model": None,
            "description": (
                (description or "")
                if description and verbose
                else f"specialized from {tag}."
            ),
        }

        # optional
        if train:
            lr_scheduler = lr_schedule or LearningRateScheduler(lr_schedule)
            train_summary = train(
                new_key,
                x_v = x_v,
                y_v = y_v,
                x_tr = x_tr,
                y_tr= y_tr,
                epochs=epochs,
                lr=lr,
                batch_size=batch_size,
                datagen=datagen,
                lr_schedule=lr_scheduler,
                resample=resample,
                iniTh=ini_th,
                step=step,
                minValPercentage=min_val_percentage,
                save_each=save_each,
                saveAboveAcc=save_above_acc,
                saveBests=save_bests,
                verbose=verbose,
            )
            print(train_summary)
        return specialist

### function : is fine, is c in, find valid pahts

In [28]:
def _is_fine_(node : TreeNode):
    return all([len(t)<=1 for t in node.refinement.valid_refinements.values()])
def _is_c_in_(node : TreeNode, t_class : int):
    return t_class in node.flat_target
    
def _find_valid_paths_(t_class : int):
    root = targetTree.nodes[0]
    path : list[TreeNode]= []
    valid_paths : list[list[TreeNode]]= []
    def dfs(node : TreeNode):
        if _is_c_in_(node,t_class):
            path.append(node)
            if _is_fine_(node):
                valid_paths.append(path.copy())
            for next_node in node.NextNodes:
                dfs(next_node)
            path.pop()
    dfs(root)
    return valid_paths

### function : get raw pred

In [29]:
def _get_raw_prediction_(k):
    if raw_predictions[k].size>0:
        return raw_predictions[k]
    else:
        print(f'model {k} is thinking...')
        model : keras.Model = models[k]
        raw_predictions[k] = model.predict(x)
        return raw_predictions[k]
        

### function : find cert part

In [30]:
def _find_certain_part_(k,th):
        """
        Find mask for certain samples for a given certainty threshold.
        
        
        Args:
            model_key: The key identifying a model in self.charts.
            predictions (np.ndarray): The raw model predictions, shape (N, #classes).
            certainty_threshold (float): The certainty threshold for max(g).
        Returns:
            np.ndarray of bool : True if model is certain on that sample w.r.t. the certainty_threshold
        """
        if raw_predictions[k].size == 0:
            p = _get_raw_prediction_(k)
        else:
            p = raw_predictions[k]
        if p.ndim == 2:
            certainty = np.max(p, axis=-1)  # shape (N,)
        elif p.ndim == 1:
            certainty = p
        certain_parts[k][th] = certainty > sigmoid(th)
        return 

### function : rho

In [31]:
def _compute_rho_(model_keys : list[tuple], thresholds : list[float] = []):
        '''
        rho indicates if model is certain about the sample given the certainty threshold.
        
        Time complexity : O(|Sample|*|keys|).
        '''
        rho : dict[tuple[tuple,float],np.ndarray] = collections.defaultdict(lambda: np.array([], dtype=float))
        e = epsilon
        for k in model_keys:
            for th in thresholds:
                if th in certain_parts[k] and certain_parts[k][th].size>0:
                    certain_part = certain_parts[k][th]
                else:
                    _find_certain_part_(k, th)
                    certain_part = certain_parts[k][th]
                rho[k, th] = np.where(certain_part, 1, e) # shape = (N,)
        return rho

### function : phi

In [32]:
def _compute_phi_(model_keys, thresholds : list[float] = []):
        '''
        phi refers to the sum of weights by overall accuracy at the certainty threshold, where each weight by accuracy is activated according to whether it's certain or not.
        
        phi maps threshold to sum of weights(overall accuracies) involving only certain models.
        rho maps (model key, threshold) to an array of floating either 1 or epsilon indicating if it's certain for the sample or not.
        '''
        phi : dict[float, np.ndarray] = collections.defaultdict(lambda: np.array([]))
        rho = _compute_rho_(model_keys, thresholds= thresholds)
        if not thresholds:
            thresholds = thresholds
        for th in thresholds:
            total_phi = np.sum([rho[model_key,th]*charts[model_key]['fuzDom'][ths_for_fuzdom[th]][0][0] for model_key in model_keys],axis=0)
            phi[th] = total_phi # shape = (N,)
        return phi

### function : weighted preds and psi

In [33]:
def _weighted_preds_and_psi_( 
            node : TreeNode,
            thresholds : list[float] = []):
        
        weighted_preds : dict[tuple,dict[float,np.ndarray]]= {}
        phi : dict[tuple,dict[float, np.ndarray]] = {}
        if not thresholds:
            thresholds = thresholds
            
        for unflattened_T in node.targets:
            model_keys = node.target_to_keys[unflattened_T]
            rho = _compute_rho_(model_keys, thresholds= thresholds)
            weighted_preds[unflattened_T] = {}
            phi[unflattened_T] = _compute_phi_(model_keys, thresholds= thresholds)
            for th in thresholds:
                # rho[k,th], phi[th] : (N,)
                # raw_predictions[k] : (N, |targets|)
                weighted_preds[unflattened_T][th] = np.sum([rho[k,th][:,np.newaxis]*charts[k]['fuzDom'][ths_for_fuzdom[th]][0][0]*raw_predictions[k] for k in model_keys],axis=0)
                weighted_preds[unflattened_T][th] = np.divide(weighted_preds[unflattened_T][th],phi[unflattened_T][th][:,np.newaxis])
                # shape = (N,|targets|)
        
        normalizer : dict[float,np.ndarray]= {}
        
        for th in thresholds:
            # psi_temp[thick_target][th] = (N,)
            normalizer[th] = np.sum([phi[thick_target][th] for thick_target in node.targets],axis=0)
        
                 
        psi : dict[tuple,dict[float,np.ndarray]]= {}
        
        for unflattened_T in node.targets:
            psi[unflattened_T] = {}
            for th in thresholds:  
                psi[unflattened_T][th] = np.divide(phi[unflattened_T][th],normalizer[th])
                # (N,) shape of floating values
        
        return weighted_preds, psi

### function :  beta

In [34]:
def _compute_beta_(
                       node : TreeNode, 
                       weighted_preds : dict[tuple,dict[float,np.ndarray]],
                       thresholds : list[float] = []):
        '''
        distribution factor at a node denoted by beta in the paper.
        from_thick_t : tuple = index tuple for the enumerated thick target stored in refinement.
        Return:
        beta maps (a thick target, valid combination) to {threshold -> N shape of floating numbers, which are distribution factor.}
        
        More precisely, it maps tuple of (thick target = tuple) and combination, which corresponds to the valid refinements contained in the thick target. The mapped value is dictionary mapping threshold to the beta, the distribution factor.
        
        E.g.
        
        [1,2,3], [4,5] with prediction values (x,y)
        [1,2], [3,4], [5] with prediction values (a,b)
        
        Refinements : [1,2] ,[3], [4], [5]
        Since [1,2,3] \cap [5] is empty, we need to distribute answer for [1,2,3]\cap[5] to other valid refinements. For [1,2,3], there are two refinements [1,2] and [3]. And distribution factor is {xa/(xa+xb)} = a/a+b from [1,2,3] to [1,2], and b/a+b from [1,2,3] to [3].
        
        
        '''
        
        beta : dict[
            tuple,dict[ # a thick target ->
                tuple[int,...],dict[ # a valid combination ->
                    float,np.ndarray] # threshold - > distribution value determined by predictions on valid refinements
                ]
            ] = collections.defaultdict(dict)
        
        preds_per_valid_c : dict[
            tuple[int,...], dict[float, np.ndarray] # a valid combination -> (th -> preds, of shape (N,))
            ] = collections.defaultdict(dict)
        
        c_contained_in_ : dict[tuple,list[tuple[int,...]]] = {}
        if not thresholds:
            thresholds = thresholds
        normalizer = {}
        for c in node.refinement.valid_refinements:
            for th in thresholds:
                preds_per_valid_c[c][th] = np.prod([weighted_preds[node.refinement.enumerated_targets[i]][th][:,j] for i, j in enumerate(c)],axis=0) # shape = (N,)
        
        for i, unflattened_T in enumerate(node.refinement.enumerated_targets):
            for j, t_ij in enumerate(unflattened_T):
                if beta[t_ij]:
                    continue
                else:
                    normalizer : dict[float,np.ndarray]= collections.defaultdict(lambda: np.zeros(len(x)))
                    c_contained_in_[t_ij] = []
                    for c , _ in node.refinement.valid_refinements.items():
                        if _ and c[i]==j: # if given valid refinement is contained in t_ij
                            c_contained_in_[t_ij].append(c)
                            beta[t_ij][c] = {}
                            for th in thresholds:
                                beta[t_ij][c][th] = preds_per_valid_c[c][th]
                                normalizer[th] += preds_per_valid_c[c][th]
                    for c in c_contained_in_[t_ij]:
                        for th in thresholds:
                            beta[t_ij][c][th] = np.divide(beta[t_ij][c][th],normalizer[th])
         
        return preds_per_valid_c, beta, c_contained_in_
        

### function : voting at node

In [115]:
def voting_at_node(node : TreeNode, thresholds : list[float]  = []):
        '''
        At a node, all appended models have the same flattening and possibly distict thick targets. We combine votes from each thick target, and distribute votes from invalid to valid refinements.
        '''
        
        
        weighted_p, psi = _weighted_preds_and_psi_(node, thresholds= thresholds)
        preds_per_valid_c , beta , c_contained_in_= _compute_beta_(node, weighted_p,thresholds=thresholds)
        n = len(x)
        
        refined_targets_temp = node.refinement.valid_refinements.values()
        refined_targets = []
        for _ in refined_targets_temp:
            if _:
                refined_targets.append(_)
        refined_targets = tuple(sorted(refined_targets))
        m = len(refined_targets)        
        
        answer : dict[float,np.ndarray] = collections.defaultdict(lambda : np.zeros((n,m), dtype=float))
        
        # Time complexity here may matter.
        # Suppose the length of targets in flattening of target at a node = K
        # Then, at worst, the number of possible combination is O(K^K). In case of 10, it's already more then 10^5.
        # Let's focus on more practical case.
        # suppose the number of target is 1000.
        # If we have 3 types of different 10 chunks of targets, then that results in 100^3.
        # So practically we may assume that there will be no drastic variety of choice in targets.
        if not thresholds:
            thresholds = thresholds
        for c in node.refinement.combinations:
            refinement = node.refinement.valid_refinements[c]
            if refinement:
                index = refined_targets.index(refinement)
                for th in thresholds:
                    answer[th][:,index] += preds_per_valid_c[c][th]
            else:
                pred_c : dict[float, np.ndarray] = collections.defaultdict(lambda : np.array((len(x),), dtype= float))
                for th in thresholds:
                    pred_c[th] = np.prod([weighted_p[node.refinement.enumerated_targets[i]][th][:,j] for i, j in enumerate(c)],axis=0)
                
                for i, j in enumerate(c):
                    for valid_c in c_contained_in_[node.refinement.enumerated_targets[i][j]]:
                        index = refined_targets.index(node.refinement.valid_refinements[c])
                        for th in thresholds:
                            answer[th][:,index] += np.prod([pred_c[th],psi[node.refinement.enumerated_targets[i]][th],beta[node.refinement.enumerated_targets[i][j]][valid_c][th]],axis=0)
        node.enumerated_refinement = refined_targets
        combined_answers[node] = answer
        return answer

### function : voting along path

In [114]:
def voting_along_a_path(path: list[TreeNode], thresholds : list[float] = []):
    
    for node in path:
        if not combined_answers[node]:
            voting_at_node(node, thresholds=thresholds)
            
    last_node : TreeNode = path[-1]
    
    enumerated_refinement = [r[0] for r in last_node.enumerated_refinement]
    m = len(enumerated_refinement)
    answers_along_path : dict[float,np.ndarray]= collections.defaultdict(lambda : np.zeros((len(x),10), dtype=float))
    
    index_seq_along_path : list[list] = []
    for i, c in enumerate(enumerated_refinement):
        index_seq_along_path.append(list())
        for node in path:
            for index, refinement in enumerate(node.enumerated_refinement):
                if c in refinement:
                    index_seq_along_path[i].append(index)
                    break
    if not thresholds:
        thresholds = thresholds
    for i in range(m):
        for th in thresholds:
            answers_along_path[th][:,enumerated_refinement[i]] = np.prod([combined_answers[node][th][:,index_seq_along_path[i][j]] for j, node in enumerate(path)], axis=0)
    path_key = tuple(path)
    combined_answers_along_a_path[path_key] = answers_along_path
    
    return answers_along_path


In [64]:
m = len(targetTree.nodes[0].flat_target)

In [63]:
def _predict_use_history_(paths : list[tuple[TreeNode,...]], thresholds : list[float]):
        
        x = small_x_v
        y = small_y_v
        n = len(x)
        answer : np.ndarray = np.zeros((n,num_classes), dtype=float)
        for c_i in range(num_classes):
            path = paths[c_i]
            threshold = thresholds[c_i]
            answer_temp = []
            for node in path:
                answer_at_node : np.ndarray = np.zeros((n,), dtype=float)
                weighted_p, psi = _weighted_preds_and_psi_(node, thresholds=[threshold])
                preds_per_valid_c , beta , _= _compute_beta_(node, weighted_p, thresholds=[threshold])
                refined_targets_temp = node.refinement.valid_refinements.items()
                # We only look at the refinement containing c_i.
                for (i, (key, value)) in enumerate(refined_targets_temp):
                    if c_i in value:
                        r = value
                        comb = key
                # We only look at the invalid refinement whose thick components have c_i.
                for c in node.refinement.combinations:
                    escape = True
                    thick_targets_containing_c_i = set()
                    for i, j in enumerate(c):
                        if c_i in node.refinement.enumerated_targets[i][j]:
                            escape = False
                            thick_targets_containing_c_i.add((i,j))
                    if escape:
                        continue
                        
                    refinement = node.refinement.valid_refinements[c]
                    
                    if refinement and refinement == r:
                        answer_at_node += preds_per_valid_c[c][threshold]
                    else: # the current refinement is invalid, but still a thick compoenet composing the refinement contains c_i, which means this invalid combination will be distributed to the refinement containing c_i.
                        pred_c = np.prod([weighted_p[node.refinement.enumerated_targets[i]][threshold][:,j] for i, j in enumerate(c)],axis=0)
                        
                        for (i,j) in thick_targets_containing_c_i:
                        
                            answer_at_node += np.prod([pred_c,psi[node.refinement.enumerated_targets[i]][threshold],beta[node.refinement.enumerated_targets[i][j]][comb][threshold]],axis=0)
                answer_temp.append(answer_at_node)
            answer[:,c_i] += np.prod(answer_temp, axis=0)
        return answer

In [61]:
def temp_pred(use_history : bool = False, thresholds : list[float] = [], optimal_paths = None, optimal_thresholds = None):
    thresholds = [0, 0.5,0.7310585786300049,0.8807970779778823,0.9525741268224334,0.9820137900379085,0.9933071490757153,0.9975273768433653,0.9990889488055994,0.9996646498695336,0.9998766054240137] if not thresholds else thresholds
    if use_history:
        return _predict_use_history_(optimal_paths,optimal_thresholds)
    else:
        for c in range(num_classes):
            paths = _find_valid_paths_(c)
            valid_paths.append(paths)
            for path in paths:
                voting_along_a_path(path,thresholds=thresholds)

In [111]:
import csv
def predict(model_keys, use_history : bool = False, predOutputFile :str = None, compute_expected_accuracy : bool = False, optimal_paths = None, optimal_thresholds = None):
        x = small_x_v
        y = small_y_v
        y_true = np.argmax(y,axis=-1)
        # targetTree = TargetTree()
        # for k in model_keys:
        #     target = charts[k]['target']
        #     key = k
        #     targetTree.add_node(
        #     target,
        #     flatTuple(target),
        #     key            
        # )
        # prediction_machine = Prediction_machine(self,x,model_keys,thresholds = thresholds, use_history=use_history)
        if use_history:
            answer = temp_pred(use_history=use_history, optimal_paths=optimal_paths, optimal_thresholds= optimal_thresholds)
            pred_label = np.argmax(answer, axis=-1)  
            accuracy = np.mean(pred_label == y_true)
            print(f"Accuracy={accuracy:.4%}")
            np.save('answer_use_history.npy',answer)
            return None, None, None, None, None
        else:
            temp_pred(thresholds=thresholds)
            if predOutputFile is None:
                predOutputFile = 'predOutputFile.csv'
            with open(predOutputFile, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["TargetClass", "PathIndex", "PathNodes", "Threshold", "Accuracy"])
                if y is not None:
                    expected_accuracy : list[dict[tuple[list[TreeNode],float],float]] = []
                    for c_i, paths_for_ci in enumerate(valid_paths):
                        expected_accuracy.append(dict())
                        print(f"\n=== Target Class {c_i} ===")
                        if not paths_for_ci:
                            print("  No valid paths found.")
                            writer.writerow([c_i, None, "No valid paths", None, None])
                            continue

                        for idx, path in enumerate(paths_for_ci):
                            print(f"  Path {idx}: {[str(node.flat_target) for node in path]}")
                            
                            # If you stored the path as a tuple in your dictionary:
                            path_key = tuple(path)
                            # (Adjust if needed. For instance, if it's a list in your dictionary, use path as-is.)

                            answers_dict = combined_answers_along_a_path.get(path_key, {})
                            if not answers_dict:
                                print("    No predictions recorded for this path.")
                                writer.writerow([c_i, idx+1, str([n.flat_target for n in path]), None, None])
                                continue

                            # Sort by threshold to present them in ascending order
                            for threshold, logits in sorted(answers_dict.items(), key=lambda x: x[0]):
                                
                                pred_label = np.argmax(logits, axis=-1)
                                
                                mask = pred_label == c_i
                                accuracy = np.sum(y_true[mask] == c_i)/np.sum(mask) if any(mask) else np.nan
                                
                                writer.writerow([
                                c_i, # target class
                                idx+1, # path index
                                str([n.flat_target for n in path]), # path nodes
                                f"{threshold:.4f}", 
                                f"{accuracy:.4%}"
                            ])
                                expected_accuracy[c_i][(tuple(path), threshold)] = accuracy
                    if compute_expected_accuracy:
                        optimal_paths = []
                        optimal_thresholds = []
                        best_value = []
                        for i, d in enumerate(expected_accuracy):
                            if not d:  
                                print(f"Expected_accuracy #{i} is empty.")
                                continue

                            # 'key=d.get' tells max() to pick the dict key whose value is largest
                            optimal_path, th = max(d, key=d.get)
                            optimal_paths.append(optimal_path)
                            optimal_thresholds.append(th)
                            best_value.append(d[(optimal_path, th)])
                        return optimal_paths, optimal_thresholds, best_value, raw_predictions, certain_parts
                    else:
                        return None, None, None, None, None
                else:
                    pass

In [102]:
charts.keys()

dict_keys([(0,), (1,), (2,), (3,)])

In [53]:

for node in targetTree.nodes:
    node._update_refinement_()
optimal_paths, optimal_thresholds, best_value, raw_predictions, certain_parts = predict(model_keys = [(0,)], compute_expected_accuracy=True)


[0, 0.5, 0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085, 0.9933071490757153, 0.9975273768433653, 0.9990889488055994, 0.9996646498695336, 0.9998766054240137]
[0, 0.5, 0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085, 0.9933071490757153, 0.9975273768433653, 0.9990889488055994, 0.9996646498695336, 0.9998766054240137]
[0, 0.5, 0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085, 0.9933071490757153, 0.9975273768433653, 0.9990889488055994, 0.9996646498695336, 0.9998766054240137]
[0, 0.5, 0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085, 0.9933071490757153, 0.9975273768433653, 0.9990889488055994, 0.9996646498695336, 0.9998766054240137]
[0, 0.5, 0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085, 0.9933071490757153, 0.9975273768433653, 0.9990889488055994, 0.9996646498695336, 0.9998766054240137]
[0, 0.5, 0.7310585786300049, 0.8807970779778823, 0.9525

In [64]:
predict(model_keys = [(0,)], use_history=True, optimal_paths=optimal_paths, optimal_thresholds=optimal_thresholds)

Accuracy=92.0000%


(None, None, None, None, None)

In [65]:
answer = np.load('answer_use_history.npy')

In [68]:
model = models[(0,)]
raw_answer = model.predict(small_x_v)

4/4 [==============================] - 0s 74ms/step


In [72]:
sum(np.argmax(raw_answer,axis=-1) == np.argmax(small_y_v,axis=-1))

92

In [74]:
base_model1 = load_model(base_models_path+'CIFAR10models/ResNet/n_3_v1_cifar10.keras')
charts[(1,)] = {
    'target': [[i] for i in range(10)],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet20v1, standard',
    'spec for loading model': None
}
models[(1,)] = base_model1
getFuzDoms((1,))

base_model2 = load_model(base_models_path+'CIFAR10models/ResNet/n_3_v2_cifar10.keras')
charts[(2,)] = {
    'target': [[i] for i in range(10)],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet20v2, standard',
    'spec for loading model': None
}
models[(2,)] = base_model2
getFuzDoms((2,))

base_model3 = load_model(base_models_path+'CIFAR10models/ResNet/n_9_v2_cifar10.keras')
charts[(3,)] = {
    'target': [[i] for i in range(10)],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet56v2, standard',
    'spec for loading model': None
}
models[(3,)] = base_model3
getFuzDoms((3,))


Computing fuzzy domains...
4/4 [==============================] - 0s 25ms/step
Computing fuzzy domains...
4/4 [==============================] - 0s 43ms/step
Computing fuzzy domains...
4/4 [==============================] - 1s 134ms/step


### Previous logifold

In [80]:
small_y_tr = to_categorical(small_y_tr,10)

In [82]:
import logifoldv1_4 as logifold
lgfd = logifold.Logifold(10,
                        name = "test_",
                        x_tr = small_x_tr,
                        y_tr = small_y_tr,
                        x_v = small_x_v,
                        y_v = small_y_v, 
                        path = 'test_/',
                        storyFile='story_')

In [83]:
lgfd.add(models[(0,)],(0,),'keras', target = [[n] for n in range(10)],description = 'Base model, Resnet56v1')
lgfd.add(models[(1,)],(1,),'keras', target = [[n] for n in range(10)],description = 'Base model, Resnet20v1')
lgfd.add(models[(2,)],(2,),'keras', target = [[n] for n in range(10)],description = 'Base model, Resnet20v2')
lgfd.add(models[(3,)],(3,),'keras', target = [[n] for n in range(10)],description = 'Base model, Resnet56v2')


Computing fuzzy domains...
The fuzzy domains have been computed.
Computing fuzzy domains...
The fuzzy domains have been computed.
Computing fuzzy domains...
The fuzzy domains have been computed.
Computing fuzzy domains...
The fuzzy domains have been computed.


In [87]:
answer = lgfd.predict(small_x_v,y=small_y_v)

Vote counting...


In [93]:
answer[4][-1][0]

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.94,0.940000,100
0.500000,0.94,0.940000,100
0.731059,0.94,0.940000,100
0.880797,0.94,0.940000,100
0.952574,0.94,0.949495,99
0.982014,0.94,0.989247,93
0.993307,0.94,1.000000,91
0.997527,0.94,1.000000,91
0.999089,0.94,1.000000,91
0.999665,0.94,1.000000,91


In [103]:
targetTree = TargetTree()
for k in [(0,),(1,),(2,),(3,)]:
    target = charts[k]['target']
    key = k
    targetTree.add_node(
    target,
    flatTuple(target),
    key            
)

In [104]:
targetTree.nodes[0].keys

[(0,), (1,), (2,), (3,)]

In [116]:

for node in targetTree.nodes:
    node._update_refinement_()
optimal_paths, optimal_thresholds, best_value, raw_predictions, certain_parts = predict(model_keys = [(0,),(1,),(2,),(3,)], compute_expected_accuracy=True)
predict(model_keys = [(0,),(1,),(2,),(3,)], use_history=True, optimal_paths=optimal_paths, optimal_thresholds=optimal_thresholds)


=== Target Class 0 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 1 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 2 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 3 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 4 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 5 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 6 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 7 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 8 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 9 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 10 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 11 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 12 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 13 ===
  Path 0: ['{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}']

=== Target Class 14 ===
  Pat

(None, None, None, None, None)

In [119]:
lgfd.turnSpecialist((0,), (0,0),[[0,1,2,3,4],[5,6,7,8,9]], epochs = 20)
lgfd.turnSpecialist((1,), (1,0),[[0,1,2],[3,4,5],[6,7,8,9]], epochs = 20)

Computing fuzzy domains...
The fuzzy domains have been computed.
compared to 0.530 in the first epoch.
The best validation accuracy was 0.620
Computing fuzzy domains...
The fuzzy domains have been computed.
compared to 0.580 in the first epoch.
The best validation accuracy was 0.770


In [120]:
answer = lgfd.predict(small_x_v,y=small_y_v)
answer[4][-1][0]

Vote counting...


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.94,0.940000,100
0.500000,0.94,0.940000,100
0.731059,0.94,0.940000,100
0.880797,0.94,0.940000,100
0.952574,0.94,0.949495,99
0.982014,0.94,0.989247,93
0.993307,0.94,1.000000,91
0.997527,0.94,1.000000,91
0.999089,0.94,1.000000,91
0.999665,0.94,1.000000,91


In [122]:
specialist_1 = load_model('model_000_000.keras')
charts[(0,0)] = {
    'target': [[0,1,2,3,4],[5,6,7,8,9]],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet20v1, standard',
    'spec for loading model': None
}
models[(0,0)] = specialist_1
getFuzDoms((0,0))

Computing fuzzy domains...
4/4 [==============================] - 1s 65ms/step


In [123]:
specialist_2 = load_model('model_001_000.keras')
charts[(1,0)] = {
    'target': [[0,1,2],[3,4,5],[6,7,8,9]],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet20v1, standard',
    'spec for loading model': None
}
models[(1,0)] = specialist_1
getFuzDoms((1,0))

Computing fuzzy domains...
4/4 [==============================] - 0s 96ms/step


In [58]:
model = load_model(dropbox_path+'model_000_000_001.keras')
charts[(0,0,1)] = {
    'target': [[n for n in range(10)],[n for n in range(10,20)]],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet56v1, trained on adv+original, and specialized to filter',
    'spec for loading model': None
}
models[(0,0,1)] = model
target = charts[(0,0,1)]['target']
targetTree.add_node(target,
                    flatTuple(target),
                    (0,0,1),
                    root=True
                    )


In [124]:
targetTree = TargetTree()
for k in [(0,),(1,),(2,),(3,),(0,0),(1,0)]:
    target = charts[k]['target']
    key = k
    targetTree.add_node(
    target,
    flatTuple(target),
    key            
)
for node in targetTree.nodes:
    node._update_refinement_()

In [127]:
targetTree.nodes[0].keys

[(0,), (1,), (2,), (3,), (0, 0), (1, 0)]

In [59]:
model = load_model(dropbox_path+'model_000_001_000.keras')
charts[(0,1)] = {
    'target': [[n] for n in range(10,20)],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet56v1, trained on adv+original, and specialized to filter',
    'spec for loading model': None
}
models[(0,1)] = model
target = charts[(0,1)]['target']
targetTree.add_node(target,
                    flatTuple(target),
                    (0,1),
                    root=False
                    )


In [155]:
answer_x = np.load('answer_use_history.npy')

In [159]:
for i in range(10):
    print(np.round(answer_x[i],3), max(answer_x[i]), np.argmax(answer_x[i]))

[0.    0.    0.    0.003 0.026 0.223 0.    0.751 0.    0.   ] 0.7513570868465549 7
[0.    0.    0.    0.008 0.    0.029 0.    0.    0.963 0.   ] 0.9634229550758998 8
[0.983 0.    0.001 0.011 0.    0.002 0.    0.    0.002 0.   ] 0.9833228925863903 0
[0.    0.    0.    0.    0.001 0.    0.94  0.    0.059 0.   ] 0.93963573343224 6
[0.    0.993 0.    0.001 0.    0.    0.    0.    0.005 0.   ] 0.9934552679459254 1
[0.    0.    0.    0.    0.    0.    0.998 0.    0.001 0.   ] 0.9977194190424633 6
[0.    0.    0.    0.002 0.    0.    0.    0.    0.997 0.   ] 0.9974107762177785 8
[0.955 0.    0.    0.    0.    0.    0.    0.    0.045 0.   ] 0.9546907991170883 0
[0.009 0.    0.016 0.016 0.004 0.    0.956 0.    0.    0.   ] 0.9559777388970058 6
[0.037 0.016 0.017 0.241 0.011 0.457 0.192 0.001 0.027 0.001] 0.45664549246430397 5


In [60]:
model = load_model(dropbox_path+'model_001_000_000.keras')
charts[(1,)] = {
    'target': [[n] for n in range(10)],
    'active': True,
    'filetype': 'keras',
    'description': 'ResNet56v1, trained on adv+original, and specialized to filter',
    'spec for loading model': None
}
models[(1,)] = model
target = charts[(1,)]['target']
targetTree.add_node(target,
                    flatTuple(target),
                    (1,),
                    root=False
                    )
